In [2]:
trained_model = "gamornet.h5"
transfer_data = "califa_s_1.0"
testing_data = "nair_abraham_2010_hdf5"

In [4]:
import tensorflow as tf
import utility.initialize_tf

1.15.3
2.2.4-tf


In [5]:
# Implementing LRN in Keras. Code from "Deep Learning with Python" by F.
# Chollet
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
class LocalResponseNormalization(Layer):

    def __init__(self, n=5, alpha=0.0001, beta=0.75, k=1.0, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x, mask=None):
        if K.image_data_format == "channels_first":
            _, f, r, c = self.shape
        else:
            _, r, c, f = self.shape
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1, 1),
                          padding="same", pool_mode="avg")
        if K.image_data_format == "channels_first":
            summed = K.sum(pooled, axis=1, keepdims=True)
            averaged = self.alpha * K.repeat_elements(summed, f, axis=1)
        else:
            summed = K.sum(pooled, axis=3, keepdims=True)
            averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom

    def get_output_shape_for(self, input_shape):
        return input_shape

In [6]:
base_model = tf.keras.models.load_model('previous_models\\' + trained_model, custom_objects={'LocalResponseNormalization': LocalResponseNormalization})
#base_model = tf.keras.models.load_model('previous_models\\' + trained_model)

#base_model = tf.saved_model.load('previous_models\\')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
base_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 42, 42, 96)        11712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 96)        0         
_________________________________________________________________
local_response_normalization (None, 21, 21, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
local_response_normalization (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 384)      

In [8]:
base_model.pop()
base_model.pop()
base_model.pop()
base_model.pop()
base_model.pop()
#base_model.pop()

In [9]:
base_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 42, 42, 96)        11712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 96)        0         
_________________________________________________________________
local_response_normalization (None, 21, 21, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
local_response_normalization (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 384)      

In [10]:
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, Flatten
# from tensorflow.keras.models import Model

# input_shape = (100, 100, 1)

# # Freeze base model
# base_model.trainable=False

# # Create New Model on top
# inputs = tf.keras.Input(shape=input_shape)
# x = base_model(inputs, training=False)

# x = Conv2D(32, kernel_size=(3, 3),
#                 activation='relu',
#                 input_shape=input_shape)(x)
# x = Conv2D(64, (3, 3), activation='relu')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)
# x = Dropout(0.25)(x)
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.5)(x)
# outputs = Dense(2, activation='softmax')(x)
# new_model = tf.keras.Model(inputs, outputs)

In [11]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Freeze base model
base_model.trainable=False

# Create New Model on top
inputs = tf.keras.Input(shape=(167, 167, 1))
x = base_model(inputs, training=False)
#x = GlobalAveragePooling2D()(x)
#x = Dense(512, activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x) # added later
outputs = Dense(2, activation='softmax')(x)
new_model = tf.keras.Model(inputs, outputs)
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 1)]     0         
_________________________________________________________________
sequential_1 (Sequential)    multiple                  3723968   
_________________________________________________________________
dense (Dense)                (None, 128)               524416    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 4,248,642
Trainable params: 524,674
Non-trainable params: 3,723,968
_________________________________________________________________


In [12]:
# Train additional layers
import numpy as np
from data_loading import *
x_dataset, y_dataset, metadata = load_hdf5_data(name=transfer_data)
x_dataset = np.expand_dims(x_dataset, axis=3)
#from sklearn import model_selection
#x_train, x_test, y_train, y_test = model_selection.train_test_split(x_dataset, y_dataset, train_size=0.7, random_state=1)

#print(x_train.shape)
#print(x_test.shape)


ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [ ]:
def create_datagen(subset):
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    datagen = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=[0.8, 1.2],
        rotation_range=45,
        fill_mode='constant',
        cval = 0,
        validation_split = 0.2
    )
    datagen.fit(subset)
    return datagen

datagen = create_datagen(x_dataset)

In [ ]:
new_model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adadelta(learning_rate=1.0),
                  metrics=['accuracy'])
train_history = new_model.fit(datagen.flow(x_dataset, y_dataset, batch_size=32), batch_size=32, epochs=1000,
          verbose=0, validation_data=datagen.flow(x_dataset, y_dataset, batch_size=32, subset='validation'))
score = new_model.evaluate(datagen.flow(x_dataset, y_dataset, batch_size=32, subset='validation'), verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from matplotlib import pyplot as plt
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
# Train whole model for fine tuning.
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
new_model.trainable = True

new_model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.1),
                  metrics=['accuracy'])
new_model.fit(datagen.flow(x_dataset, y_dataset, batch_size=32), batch_size=32, epochs=10,
          verbose=1, validation_data=datagen.flow(x_dataset, y_dataset, batch_size=32, subset='validation'), callbacks=[callback])
score = new_model.evaluate(datagen.flow(x_dataset, y_dataset, batch_size=32, subset='validation'), verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])



In [ ]:
new_model.save(r"saved_models\gamornet_transfer_learning.h5", overwrite=True, save_format='h5')

In [ ]:
x, Y, metadata = load_hdf5_data(name=testing_data)

pred = new_model.predict(x).argmax(axis=1)

results = np.asarray(np.unique(pred, return_counts=True)).T
print("total images: " + str(np.sum(results)-1))
print("Percentage of E: ", results[0, 1]/(np.sum(results)-1)*100)
print("Percentage of ES: ", results[1, 1]/(np.sum(results)-1)*100)

In [ ]:
# from sklearn import metrics

# true_data = y_test
# predicted_data = new_model.predict(x_test)
# metrics.ConfusionMatrixDisplay(metrics.confusion_matrix(true_data.argmax(axis=1), predicted_data.argmax(axis=1))).plot(cmap='Blues')

In [ ]:
# from sklearn import metrics
# class_labels = np.unique(metadata["class"])
# print(class_labels)
# recid = -1
# print(metadata[recid])

# #print(class_labels[Y_val[recid]])
# print(Y_val[recid])
# #print(predicted_data)
# true_data = Y_val
# predicted_data = new_model.predict(x_val)
# metrics.ConfusionMatrixDisplay(metrics.confusion_matrix(true_data.argmax(axis=1), predicted_data.argmax(axis=1)),
#                                display_labels=class_labels).plot(cmap='Blues')

In [ ]:
# #mpl.rcParams['figure.figsize'] = [16, 40]
# #mpl.rcParams['figure.dpi'] = 72
# plotting_helpers.plot_classification_results(images=x_val, display_size=(12, 5), y_preds=predicted_data.argmax(axis=1), y_trues=Y_val.argmax(axis=1),
#                             y_labels=class_labels,
#                             galaxy_names=metadata["name"], random_sample=True)

In [ ]:
# no_arg = new_model.predict(x_dataset[:1])
# no_arg.argmax(axis=1)

In [ ]:
# print(x_val.shape)
# pyplot.imshow(x_val[10])
# pyplot.colorbar()
# pyplot.show()